In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
dfHarm = pd.read_csv("harmToGraph.csv",sep=";")

dfPrices = pd.read_csv("prices.csv",sep=";")

foodBR = set(dfPrices["food"].to_list()[0:4])

print(foodBR)


{'Pastilha Menta Zero Açúcar Trident Mints Lata 22,5g 50 Unidades', 'Pastilha Morango Zero Açúcar Trident Mints Lata 22,5g 50 Unidades', 'Sal para Churrasco Himalaia Tradicional BR SPICES 300g', 'Goma de Mascar MENTOS PEPPERMINT 35G'}


In [7]:
f1 = dfHarm["food1"].to_list()
f2 = dfHarm["food2"].to_list()
amount  = dfHarm["amount"].to_list()
harmDict = dict()
i = 0
while i < len(f1):
	harmDict[f1[i]+ " && " +f2[i]] = amount[i]
	i+=1
print(harmDict[f1[1]+ " && " +f2[1]])

0.200008


In [8]:
def harmMean(table):
	amountTable = []
	i = 0
	while i < len(table):
		j = i+1
		while j < len(table):
			if (table[j]+ " && " +table[i]) in harmDict.keys():
				amountTable.append(harmDict[table[j]+ " && " +table[i]])
			elif (table[i]+ " && " +table[j]) in harmDict.keys():
				amountTable.append(harmDict[table[i]+ " && " +table[j]])
			j+=1
		i+=1
	if len(amountTable) == 0:
		return 0
	else:
		return sum(amountTable)/len(amountTable)

In [9]:
"""
A set partitioning model of a wedding seating problem

Authors: Stuart Mitchell 2009
"""

max_tables = 5
max_table_size = 4
guests = "finikia;cajun prime rib roast;korean chicken wings;rajma masaledar;savory polenta;moong soup;chana ghassi;".split(";")




# create list of all possible tables
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, max_table_size)]

# create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts(
    "table", possible_tables, lowBound=0, upBound=1, cat=pulp.LpInteger
)

seating_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

seating_model += pulp.lpSum([harmMean(table) * x[table] for table in possible_tables])

# specify the maximum number of tables
seating_model += (
    pulp.lpSum([x[table] for table in possible_tables]) <= max_tables,
    "Maximum_number_of_tables",
)

# A guest must seated at one and only one table
for guest in guests:
    seating_model += (
        pulp.lpSum([x[table] for table in possible_tables if guest in table]) == 1,
        "Must_seat_%s" % guest,
    )

seating_model.solve()

print("The choosen tables are out of a total of %s:" % len(possible_tables))
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)

C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyboardInterrupt: 